In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os

# Predefined angle data for each class
class_angles = {
    'hachijidachijodanyoko': [
        [178.6704642, 121.4324436, 178.6704642, 121.4324436, 176.2882244, 176.2882244, 179.0472569, 155.9306886, 174.7958128],
        [74.73670094, 179.6333059, 74.73670094, 179.6333059, 179.2610734, 179.2610734, 177.7071013, 157.4409467, 146.1440207]
    ],
    'sanchindachiageuke': [
        [117.6311735, 177.122714, 117.6311735, 177.122714, 172.4470531, 172.4470531, 179.84531, 178.0124464, 154.3915227],
        [88.35833, 130.8359801, 88.35833, 130.8359801, 178.8655222, 178.8655222, 179.2608471, 159.6653805, 172.8783815]
    ],
    'sanchindachijodantsuki': [
        [79.67428505, 117.6143462, 79.67428505, 117.6143462, 175.4425771, 175.4425771, 179.9083308, 179.6819875, 162.7090183],
        [132.2753167, 61.93098373, 132.2753167, 61.93098373, 177.3881788, 177.3881788, 177.6381035, 156.7447632, 179.4797426]
    ],
    'sanchindachisotouke': [
        [11.47716376, 114.5169405, 11.47716376, 114.5169405, 176.3834111, 176.3834111, 175.592297, 179.2147185, 163.3744971],
        [87.20938557, 16.70051146, 87.20938557, 16.70051146, 178.365065, 178.365065, 174.4879716, 162.4809264, 176.8140283]
    ],
    'shikodachigedanbarai': [
        [176.4795284, 153.3314441, 176.4795284, 153.3314441, 107.3410318, 107.3410318, 97.62956052, 153.8734055, 156.7703905],
        [147.1951634, 175.074177, 147.1951634, 175.074177, 103.3349196, 103.3349196, 128.5222031, 157.1361985, 145.8509965]
    ],
    'sotoukemaegeri': [
        [141.6715664, 12.61353582, 141.6715664, 12.61353582, 177.0001987, 177.0001987, 175.727286, 164.0403825, 160.4981336],
        [15.39189415, 84.51329129, 15.39189415, 84.51329129, 174.1109382, 174.1109382, 158.9654381, 178.2361718, 159.3976762]
    ],
    'zenkutsudachiawasetsuki': [
        [115.8580351, 130.7511613, 115.8580351, 130.7511613, 171.9446945, 171.9446945, 172.6747746, 144.105041, 178.9400008]
    ],
    'zenkutsudachichudantsuki': [
        [115.8580351, 130.7511613, 115.8580351, 130.7511613, 171.9446945, 171.9446945, 172.6747746, 144.105041, 178.9400008],
        [156.0357435, 61.09435279, 156.0357435, 61.09435279, 172.3895637, 172.3895637, 162.5085944, 157.2766185, 159.0072517]
    ],
    'zenkutsudachiempiuke': [
        [5.537744135, 138.5696075, 5.537744135, 138.5696075, 176.3914556, 176.3914556, 173.9572285, 166.2690671, 157.8046774],
        [156.0357435, 61.09435279, 156.0357435, 61.09435279, 172.3895637, 172.3895637, 162.5085944, 157.2766185, 159.0072517]
    ]
}

# Mapping from class indexes to class names
class_index_to_name = {
    0: 'hachijidachijodanyoko',
    1: 'sanchindachiageuke',
    2: 'sanchindachijodantsuki',
    3: 'sanchindachisotouke',
    4: 'shikodachigedanbarai',
    5: 'sotoukemaegeri',
    6: 'zenkutsudachiawasetsuki',
    7: 'zenkutsudachichudantsuki',
    8: 'zenkutsudachiempiuke'
}

def load_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

# Function to extract 3D keypoints using Mediapipe
def extract_keypoints(image):
    mp_pose = mp.solutions.pose
    with mp_pose.Pose(static_image_mode=True) as pose:
        image_resized = cv2.resize(image, (512, 384))
        results = pose.process(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))
        if not results.pose_landmarks:
            return None
        keypoints = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.pose_landmarks.landmark], dtype=np.float32)
    return keypoints

# Function to classify pose using a loaded model and keypoints
def classify_pose(model, keypoints):
    prediction = model.predict(np.expand_dims(keypoints.flatten(), axis=0))
    class_index = np.argmax(prediction)
    class_name = class_index_to_name[class_index]
    return class_name

def calculate_angle(a, b, c):
    # Calculate the angle between three points
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

def extract_angles(keypoints):
    if keypoints is None:
        return None
    
    LEFT_SHOULDER = mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value
    RIGHT_SHOULDER = mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value
    LEFT_ELBOW = mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value
    RIGHT_ELBOW = mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value
    LEFT_WRIST = mp.solutions.pose.PoseLandmark.LEFT_WRIST.value
    RIGHT_WRIST = mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value
    LEFT_HIP = mp.solutions.pose.PoseLandmark.LEFT_HIP.value
    RIGHT_HIP = mp.solutions.pose.PoseLandmark.RIGHT_HIP.value
    LEFT_KNEE = mp.solutions.pose.PoseLandmark.LEFT_KNEE.value
    RIGHT_KNEE = mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value
    LEFT_ANKLE = mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value
    RIGHT_ANKLE = mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value
    LEFT_HEEL = mp.solutions.pose.PoseLandmark.LEFT_HEEL.value
    RIGHT_HEEL = mp.solutions.pose.PoseLandmark.RIGHT_HEEL.value

    angles = [
        calculate_angle(keypoints[LEFT_HIP], keypoints[LEFT_SHOULDER], keypoints[LEFT_ELBOW]),
        calculate_angle(keypoints[RIGHT_HIP], keypoints[RIGHT_SHOULDER], keypoints[RIGHT_ELBOW]),
        calculate_angle(keypoints[LEFT_SHOULDER], keypoints[LEFT_ELBOW], keypoints[LEFT_WRIST]),
        calculate_angle(keypoints[RIGHT_SHOULDER], keypoints[RIGHT_ELBOW], keypoints[RIGHT_WRIST]),
        calculate_angle(keypoints[LEFT_SHOULDER], keypoints[LEFT_HIP], keypoints[LEFT_KNEE]),
        calculate_angle(keypoints[LEFT_HIP], keypoints[LEFT_KNEE], keypoints[LEFT_ANKLE]),
        calculate_angle(keypoints[RIGHT_HIP], keypoints[RIGHT_KNEE], keypoints[RIGHT_ANKLE]),
        calculate_angle(keypoints[LEFT_KNEE], keypoints[LEFT_ANKLE], keypoints[LEFT_HEEL]),
        calculate_angle(keypoints[RIGHT_KNEE], keypoints[RIGHT_ANKLE], keypoints[RIGHT_HEEL])
    ]

    return angles

def calculate_similarity(extracted_angles, predefined_angles):
    if len(extracted_angles) != len(predefined_angles):
        return 0.0
    diff = np.abs(np.array(extracted_angles) - np.array(predefined_angles))
    similarity = 100 - np.mean(diff)
    return similarity

def plot_keypoints_3d(keypoints):
    # Define the connections between keypoints
    connections = [
        (0, 1), (1, 2), (2, 3), (3, 7),
        (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),
        (11, 12), (11,13), (13, 15), (15, 21),(15, 17), (15, 19),  (17, 19),
        (12, 14), (14, 16), (16,22), (16,18), (16,20), (18,20),
        (11,23),(23, 25), (25,27), (27,29), (27,31), (29,31),
        (12, 24), (24, 26), (26, 28), (28,30), (28, 32), (30, 32)
    ]
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Extract x, y, z coordinates
    x = -keypoints[:, 2]
    y = -keypoints[:, 0]
    z = -keypoints[:, 1]
    
    ax.scatter(x, y, z, c='r', marker='o')
    
    for connection in connections:
        x_coords = [x[connection[0]], x[connection[1]]]
        y_coords = [y[connection[0]], y[connection[1]]]
        z_coords = [z[connection[0]], z[connection[1]]]
        ax.plot(x_coords, y_coords, z_coords, 'b')
    
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    
    plt.show()



def main_folder(folder_path, model_path):
    model = load_model(model_path)
    final_answer = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"): 
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            keypoints = extract_keypoints(image)
            if keypoints is None:
                print(f"No keypoints detected for {filename}.")
                continue
            class_name = classify_pose(model, keypoints)
            extracted_angles = extract_angles(keypoints)

            print(f"File: {filename}")
            print(f"Extracted Angles: {extracted_angles}")

            if class_name in class_angles:
                predefined_angles = class_angles[class_name][0]  # Use the first set of angles for comparison
                print(f"Predefined Angles for {class_name}: {predefined_angles}")
                similarity = calculate_similarity(extracted_angles, predefined_angles)
                print(f"Classified as: {class_name}")
                print(f"Similarity with predefined angles: {similarity:.2f}%")

                final_answer = (final_answer+similarity)/100*9
            else:
                print(f"Class {class_name} does not have predefined angles.")
            print("--------------------")
    if final_answer<6.0:
        print(f"Your final score {final_answer:.2}. You Should refer guide videos again... Let's do It")
    elif final_answer<7.0:
        print(f"Your final score {final_answer:.2}. You are good at this Kata. But you can develop more. Let's do it")
    elif final_answer<9.0:
        print(f"Your final score {final_answer:.2}. Congradulations !!!..You are really good at this Kata")
    else:
        print(f"Your final score is not withing the range. Sorry to Inform that. Please try again...")

# Example usage for a folder of images
folder_path = 'test_folder'
model_path = 'model.h5'
main_folder(folder_path, model_path)



1/1 [==============================] - 0s 96ms/step
File: V2_frame_1008.png
Extracted Angles: [63.27058, 62.721195, 65.108925, 94.38538, 132.56972, 155.27344, 115.71731, 165.97119, 150.1299]
Predefined Angles for sotoukemaegeri: [141.6715664, 12.61353582, 141.6715664, 12.61353582, 177.0001987, 177.0001987, 175.727286, 164.0403825, 160.4981336]
Classified as: sotoukemaegeri
Similarity with predefined angles: 52.74%
--------------------
1/1 [==============================] - 0s 33ms/step
File: V2_frame_1091.png
Extracted Angles: [46.30714, 83.18787, 83.646034, 72.75548, 132.10931, 88.144936, 143.18102, 149.76964, 157.7764]
Predefined Angles for sotoukemaegeri: [141.6715664, 12.61353582, 141.6715664, 12.61353582, 177.0001987, 177.0001987, 175.727286, 164.0403825, 160.4981336]
Classified as: sotoukemaegeri
Similarity with predefined angles: 48.07%
--------------------
1/1 [==============================] - 0s 33ms/step
File: V2_frame_327.png
Extracted Angles: [141.91798, 10.493475, 154.213